In [ ]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "")
)
es.ping()
# true if connection is successfull

True

## Prepare the data

In [4]:
# convert the relevant fields to vectors via BERT model
import pandas as pd
df = pd.read_csv("./myntra_products_catalog.csv").loc[:499]
df.head()

/var/folders/q8/w1v60rz107qfsqsys_y9z0gr0000gr/T/ipykernel_86170/1908251434.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [6]:
df.fillna("None", inplace=True)

## Convert the Relevant field to vector using BERT model

In [7]:
# you can refer https://sbert.net/docs/pretrained_models.html
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

ModuleNotFoundError: No module named 'sentence_transformers'